# 시카고 샌드위치 맛집 분석

## 1. 시카고 샌드위치 맛집 사이트에 접근하기

In [84]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [85]:
url_base="https://www.chicagomag.com"
url_sub ="/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/"
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')


In [86]:
rest_list = soup.find_all(class_='sammy')
rest_list[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

In [87]:
len(rest_list)

50

In [88]:
a = rest_list[0].find_all('a')
a

[<a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
 Old Oak Tap<br>
 <em>Read more</em> </br></br></a>]

## 2. 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [89]:
tmp_one = rest_list[0]
tmp_one.find(class_='sammyRank').get_text()

'1'

In [90]:
tmp_one.find(class_='sammyListing').get_text()

'BLT\r\nOld Oak Tap\nRead more '

In [91]:
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [92]:
tmp_str=tmp_one.find(class_='sammyListing').get_text()
tmp_str.split('\n')

['BLT\r', 'Old Oak Tap', 'Read more ']

In [93]:
tmp_res = tmp_str.split('\n')
menu = tmp_res[0].replace('\r','')
cafe = tmp_res[1]
print(menu, cafe)

BLT Old Oak Tap


In [94]:
import re

In [95]:
re.split(('\n|\r\n'), tmp_str)

['BLT', 'Old Oak Tap', 'Read more ']

In [96]:
print(re.split(('\n|\r\n'), tmp_str)[0], re.split(('\n|\r\n'), tmp_str)[1])

BLT Old Oak Tap


In [97]:
from urllib.parse import urljoin

In [175]:
## 데이터 프레임 만들기 전에 항목 데이터 구하기

rank =[]
main_menu = [] 
cafe_name = []
url_add =[]
count=0
for rest in rest_list:
    rank.append(int(rest.find(class_="sammyRank").get_text()))
    tmp_str = rest.find(class_='sammyListing').get_text()
    main_menu.append(re.split(('\n|\r\n'), tmp_str)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_str)[1])
    url_add.append(urljoin(url_base, rest.find('a')['href']))
    count+=1
count

50

In [99]:
print(main_menu[:5])

['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']


In [100]:
print(cafe_name[:5])

['Old Oak Tap', 'Au Cheval', 'Xoco', 'Al’s Deli', 'Publican Quality Meats']


In [101]:
print(url_add[-5:])

['http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Chickpea-Kufta/', 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-The-Goddess-and-Grocer-Debbies-Egg-Salad/', 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Zenwich-Beef-Curry/', 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Toni-Patisserie-Le-Vegetarien/', 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Phoebes-Bakery-The-Gatsby/']


In [102]:
import pandas as pd
df = pd.DataFrame({'rank':rank, 'cafe':cafe_name,'menu':main_menu, 'url':url_add})
df.head()

,rank,cafe,menu,url
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [103]:
#데이터 프레임 csv로 저장
df.to_csv('chicago1.csv',sep=',', encoding='utf-8')

## 3. 다수의 웹 페이지에 자동으로 접근해서원하는 정보 가져오기

In [104]:
df['url'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [105]:
url_sub = tmp_one.find('a')['href']

In [106]:
url_sub

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [125]:
html = urlopen(df['url'][0])
soup = BeautifulSoup(html, 'html.parser')

In [126]:
info_list = soup.find(class_='addy').get_text()
info_list

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [127]:
info_list.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [128]:
tmp_price = info_list.split()[0][:-1] #마지막 글자 .을 제외한([:-1])
tmp_price

'$10'

In [129]:
info_list.split()[1:-2]

# -2 전까지 가져와라 라고 하면 된다*****  그 뒤에 정보 두개는 전화번호와 url

['2109', 'W.', 'Chicago', 'Ave.,']

In [130]:
' '.join(info_list.split()[1:-2])
#여러개의 리스트가 하나의 스트링으로 합쳐지는 것*****

'2109 W. Chicago Ave.,'

In [131]:
tmp_addr = ' '.join(info_list.split()[1:-2])[:-1]
#조각내진 주소를 하나의 스트링으로 합쳐지고 마지막에 있는 , 가지 없애기

In [136]:

address = [] 
price = []

for rest in info_list:
    price.append(rest.split()[0][:-1])
    address.append(' '.join(rest.split()[1:-2][:-1]))

IndexError: list index out of range

## - 상태 진행비 적용

In [39]:
from tqdm import tqdm_notebook
import time

In [40]:
price = []
addr = []
for n in tqdm_notebook(df.index):
    html = urlopen(df['url'][n])
    soup_tmp = BeautifulSoup(html, 'html.parser')
    tmp_str = soup_tmp.find(class_='addy').get_text()
    
    price.append(tmp_str.split()[0][:-1])
    addr.append(' '.join(tmp_str.split()[1:-2][:-1]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [133]:
del df['url']

In [134]:
df['price'] = price

ValueError: Length of values does not match length of index

In [137]:
df['address'] = addr

In [138]:
df.to_csv('chicago2.csv',sep=',', encoding='utf-8')

In [139]:
df

,rank,cafe,menu,address
0,1,Old Oak Tap,BLT,2109 W. Chicago
1,2,Au Cheval,Fried Bologna,800 W. Randolph
2,3,Xoco,Woodland Mushroom,445 N. Clark
3,4,Al’s Deli,Roast Beef,"914 Noyes St.,"
4,5,Publican Quality Meats,PB&L,825 W. Fulton
5,6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,100 E.
6,7,Acadia,Lobster Roll,1639 S. Wabash
7,8,Birchwood Kitchen,Smoked Salmon Salad,2211 W. North
8,9,Cemitas Puebla,Atomica Cemitas,3619 W. North
9,10,Nana,Grilled Laughing Bird Shrimp and Fried Po’ Boy,3267 S. Halsted


In [140]:
df.set_index('rank',inplace=True)
df.head()

,cafe,menu,address
rank,,,
1,Old Oak Tap,BLT,2109 W. Chicago
2,Au Cheval,Fried Bologna,800 W. Randolph
3,Xoco,Woodland Mushroom,445 N. Clark
4,Al’s Deli,Roast Beef,"914 Noyes St.,"
5,Publican Quality Meats,PB&L,825 W. Fulton


In [141]:
df.to_csv('chicago2.csv',sep=',', encoding='utf-8')

## 3. 맛집 위치를 지도에 표기하기

In [154]:
import folium
import googlemaps
import pandas as pd

In [155]:
key_fd = open('googlemapskey.txt', mode = 'r')
gmaps_key = key_fd.read(100)
key_fd.close()

In [156]:
gmaps = googlemaps.Client(key=gmaps_key)

In [161]:
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['address'][n] != 'Multipl':
        target_name = df['address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lat'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [163]:
df['lat']=lat
df['lng']=lng
df.head()

,cafe,menu,address,lat,lng
rank,,,,,
1,Old Oak Tap,BLT,2109 W. Chicago,41.895605,41.895605
2,Au Cheval,Fried Bologna,800 W. Randolph,41.884658,41.884658
3,Xoco,Woodland Mushroom,445 N. Clark,41.890523,41.890523
4,Al’s Deli,Roast Beef,"914 Noyes St.,",42.058914,42.058914
5,Publican Quality Meats,PB&L,825 W. Fulton,41.886690,41.886690


In [165]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()],
                    zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()],
             popup='center').add_to(mapping)
mapping

In [169]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()],
                    zoom_start=11)

for n in df.index:
    if df['address'][n] != 'Multipl':
        folium.Marker([df['lat'][n], df['lng'][n]],
                     popup=df['cafe'][n]).add_to(mapping)
mapping

In [174]:
mapping.save('chicago.html')